In [37]:
import json
import csv

## Simplification des fichiers TCL initiaux

In [42]:
# lire fichier json
dataArret = {}
with open('data/sytral_tcl_sytral.tclarret.json') as json_file:
    dataArret = json.load(json_file)

dataLigne = {}
with open('data/sytral_tcl_sytral.tcllignemetrofuni_2_0_0.json') as json_file:
    dataLigne = json.load(json_file)
    
dataLigneTram = {}
with open('data/sytral_tcl_sytral.tcllignetram_2_0_0.json') as json_file:
    dataLigneTram = json.load(json_file)


In [43]:
lignes= set()
for ligne in dataLigne['features']:
    lignes.add((ligne['properties']["code_ligne"],ligne['properties']["ligne"]))
for ligne in dataLigneTram['features']:
    lignes.add((ligne['properties']["code_ligne"],ligne['properties']["ligne"]))

In [44]:
# supprimer les lignes qui ne sont pas dans lignes
dataArretcopy= dataArret.copy()
dataArret['features']=[]
for arret in dataArretcopy['features']:
    add=False
    for transport in arret['properties']['desserte'].split(','):
        for ligne in lignes:
            if transport.split(':')[0] == ligne[0]:
                add=True
                # si nom_ligne n'existe pas, on le crée
                if 'nom_ligne' not in arret['properties']:
                    arret['properties']['nom_ligne']=set()
                arret['properties']['nom_ligne'].add(ligne[1])
    if add:
        arret['properties']['nom_ligne']=list(arret['properties']['nom_ligne'])
        dataArret['features'].append(arret)
                
                

In [45]:
with open('data/sytral_tcl_sytral.tclarret_sans_bus.json', 'w') as outfile:
    json.dump(dataArret, outfile)

## Conversion fichier frequentation en csv

In [111]:
# Lecture du fichier de fréquentation
with open('data/frequentationTCL.txt', encoding='utf-8') as f:
    dataFrequentation = f.readlines()

dataFrequentation = [x.strip() for x in dataFrequentation]


In [112]:
# Parsing du fichier de fréquentation
data = {}
l_stations = []
for stations in dataFrequentation:
    if stations == "": continue
    else:
        lig = stations.split(':')
        lig[0] = lig[0].strip()
        lig[1] = lig[1].strip()
        if lig[0].replace(' ','')=="Ligne":
            ligne=lig[1]
            data[ligne]={}
        elif lig[0].replace(' ','')=="Total":
            data[ligne]["Total"]=lig[1]
        else:
            data[ligne][lig[0]]=lig[1]
            l_stations.append(lig[0])


In [113]:
with open('data/frequentationTCL.csv', 'w', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["ligne","arret","frequentation"])
    for ligne in data:
        for arret in data[ligne]:
            if arret!="Total":
                writer.writerow([ligne,arret,data[ligne][arret]])

## Ajout des fréquentations dans le fichier des arrêts	

In [114]:
dataArret = {}
with open('data/sytral_tcl_sytral.tclarret_sans_bus.json') as json_file:
    dataArret = json.load(json_file)

# Ajout de la fréquentation dans le fichier json
for arret in dataArret['features']:
    if arret['properties']['nom'].strip() in l_stations or arret['properties']['nom_ligne'][0] in ['F1','F2','T6','T7']:
        # arret['properties']['frequentation']=data[arret['properties']['nom']]['Total']
        pass
    else:
        print(arret['properties']['nom'], arret['properties']['nom_ligne'])
        arret['properties']['frequentation']=0  

Oullins Centre ['B']
Oullins Centre ['B']
St-Genis-Laval Hôp. Sud ['B']
St-Genis-Laval Hôp. Sud ['B']
